## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import optuna
import time
import lightgbm as lgb
import os, json
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import root_mean_squared_error
from scipy.optimize import minimize as sp_minimize
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

d:\Program Files\conda_envs\nitor_kuas\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configs

In [2]:
train_path  = '../data/train.csv'
test_path   = '../data/test_for_participants.csv'
sample_path = '../data/sample_submission.csv'

VAL_PHYSICS_START = '2024-09-01'
VAL_PHYSICS_END   = '2025-01-01'
VAL_RECENCY_START = '2025-06-01'

SEED     = 42
N_TRIALS = 25

os.makedirs('../models/v14/', exist_ok=True)
os.makedirs('../submissions/',  exist_ok=True)
np.random.seed(SEED)

## Data Loading

In [3]:
train_raw  = pd.read_csv(train_path)
test_raw   = pd.read_csv(test_path)
sample_sub = pd.read_csv(sample_path)

for dfi in [train_raw, test_raw]:
    dfi['delivery_start'] = pd.to_datetime(dfi['delivery_start'])
    dfi['delivery_end']   = pd.to_datetime(dfi['delivery_end'])

train_raw['is_test'] = 0
test_raw['is_test']  = 1
test_raw['target']   = np.nan

df = pd.concat([train_raw, test_raw], ignore_index=True)
df = df.sort_values(['market', 'delivery_start']).reset_index(drop=True)
df.head()

,id,target,market,global_horizontal_irradiance,diffuse_horizontal_irradiance,direct_normal_irradiance,cloud_cover_total,cloud_cover_low,cloud_cover_mid,cloud_cover_high,...,wind_speed_80m,wind_direction_80m,wind_gust_speed_10m,wind_speed_10m,solar_forecast,wind_forecast,load_forecast,delivery_start,delivery_end,is_test
0,0,-1.913,Market A,0.0,0.0,0.0,2.0,0.0,0.0,2.0,...,31.253719,245.501450,25.199999,15.077082,0.0,24050.1,38163.0100,2023-01-01 00:00:00,2023-01-01 01:00:00,0
1,5,-0.839,Market A,0.0,0.0,0.0,15.0,0.0,0.0,15.0,...,30.918108,242.241547,23.400000,14.186923,0.0,23886.3,37379.1898,2023-01-01 01:00:00,2023-01-01 02:00:00,0
2,10,-1.107,Market A,0.0,0.0,0.0,17.0,0.0,0.0,17.0,...,26.983196,224.999893,21.240000,12.413477,0.0,23366.5,36336.8303,2023-01-01 02:00:00,2023-01-01 03:00:00,0
3,15,0.035,Market A,0.0,0.0,0.0,16.0,0.0,0.0,16.0,...,22.218153,229.600174,16.199999,10.483357,0.0,22829.8,35337.7595,2023-01-01 03:00:00,2023-01-01 04:00:00,0
4,20,-0.829,Market A,0.0,0.0,0.0,10.0,0.0,0.0,10.0,...,27.210381,244.113022,18.359999,11.918120,0.0,22347.6,34474.3403,2023-01-01 04:00:00,2023-01-01 05:00:00,0


## Feature Engineering — Base & Market Encoding

In [4]:
# ── Basic time features ──────────────────────────────────────────────────────
ds = df['delivery_start']
df['hour']         = ds.dt.hour
df['day_of_week']  = ds.dt.dayofweek
df['day_of_month'] = ds.dt.day
df['month']        = ds.dt.month
df['quarter']      = ds.dt.quarter
df['day_of_year']  = ds.dt.dayofyear
df['year']         = ds.dt.year
df['is_weekend']   = (ds.dt.dayofweek >= 5).astype(np.int8)
df['week_of_year'] = ds.dt.isocalendar().week.astype(int)

# Cyclical encoding
df['hour_sin']        = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos']        = np.cos(2 * np.pi * df['hour'] / 24)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['month_sin']       = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos']       = np.cos(2 * np.pi * df['month'] / 12)
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

# Market encoding
market_map = {f'Market {c}': i for i, c in enumerate('ABCDEF')}
df['market_id'] = df['market'].map(market_map).astype(np.int8)

# Advanced demand and supply features
df['residual_demand']       = df['load_forecast'] - df['solar_forecast'] - df['wind_forecast']
df['supply_ratio']          = (df['solar_forecast'] + df['wind_forecast']) / (df['load_forecast'] + 1)
df['renewable_ratio']       = (df['solar_forecast'] + df['wind_forecast']) / (df['solar_forecast'] + df['wind_forecast'] + df['load_forecast'] + 1)
df['net_supply']            = df['solar_forecast'] + df['wind_forecast']
df['demand_supply_balance'] = df['load_forecast'] / (df['solar_forecast'] + df['wind_forecast'] + 1)

# Tightness ratios
df['tightness_ratio']  = df['residual_demand'] / (df['load_forecast'] + 1)
df['tightness_x_month']= df['tightness_ratio'] * df['month']
df['tightness_x_hour'] = df['tightness_ratio'] * df['hour']
df['tightness_x_dow']  = df['tightness_ratio'] * df['day_of_week']

# Price sensitivity
df['solar_wind_ratio'] = df['solar_forecast'] / (df['wind_forecast'] + 1)
df['wind_solar_ratio'] = df['wind_forecast']  / (df['solar_forecast'] + 1)

## Feature Engineering — Advanced Weather Physics

In [5]:
# ── Advanced Weather Physics Features ────────────────────────────────────────
if 'convective_available_potential_energy' in df.columns:
    df['cape'] = df['convective_available_potential_energy']
if 'precipitation_amount' in df.columns:
    df['precipitation'] = df['precipitation_amount']
if 'apparent_temperature_2m' in df.columns:
    df['apparent_temperature'] = df['apparent_temperature_2m']
if 'freezing_level_height' in df.columns:
    df['boundary_layer_height'] = df['freezing_level_height']

# Saturation vapour pressure (Tetens formula)
es = 6.112 * np.exp((17.67 * df['air_temperature_2m']) / (df['air_temperature_2m'] + 243.5))
ea = (df['relative_humidity_2m'] / 100.0) * es
df['vapour_pressure_deficit_2m'] = es - ea
df['precipitation_probability']  = np.where(df['relative_humidity_2m'] > 85, 50, 0)

df['dew_point_depression']    = df['air_temperature_2m'] - df['dew_point_temperature_2m']
df['wet_bulb_depression']     = df['air_temperature_2m'] - df['wet_bulb_temperature_2m']
df['humidity_ratio']          = (0.622 * df['vapour_pressure_deficit_2m']) / (df['surface_pressure'] - df['vapour_pressure_deficit_2m'])
df['blh_normalized_pressure'] = df['boundary_layer_height'] / (df['surface_pressure'] / 1000)

# Wind shear
df['wind_shear']       = df['wind_speed_80m'] - df['wind_speed_10m']
df['wind_shear_ratio'] = df['wind_speed_80m'] / (df['wind_speed_10m'] + 0.1)

# Convection indices
df['cape_cin_interaction']          = df['cape'] * df['convective_inhibition']
df['convection_potential']          = df['cape'] / (abs(df['convective_inhibition']) + 1)
df['visibility_cloud_interaction']  = df['visibility'] / (df['cloud_cover_total'] + 1)

# Combined weather severity
df['weather_severity'] = (
    df['cloud_cover_total'] / 100 +
    (100 - df['visibility'].clip(0, 100)) / 100 +
    df['precipitation_probability'] / 100 +
    df['cape'] / 1000
) / 4

# Solar / wind potential
df['solar_potential'] = df['global_horizontal_irradiance'] * (1 - df['cloud_cover_total'] / 100)
df['wind_potential']  = df['wind_speed_80m'] ** 3

# Extreme weather flags (only extreme_wind kept; extreme_temp/extreme_precip will be pruned)
df['extreme_temp']   = ((df['air_temperature_2m'] > 30) | (df['air_temperature_2m'] < -5)).astype(int)
df['extreme_wind']   = (df['wind_speed_80m'] > 25).astype(int)
df['extreme_precip'] = (df['precipitation'] > 5).astype(int)

# Seasonal x weather interactions
df['temp_month_interaction'] = df['air_temperature_2m'] * df['month']
df['wind_month_interaction'] = df['wind_speed_80m']     * df['month']
df['solar_hour_interaction'] = df['solar_forecast']     * df['hour']

# Degree-hours
df['cooling_degree_hours'] = np.maximum(df['air_temperature_2m'] - 22, 0)
df['heating_degree_hours'] = np.maximum(18 - df['air_temperature_2m'], 0)

# VPD normalised
df['vpd_normalized'] = df['vapour_pressure_deficit_2m'] / df['surface_pressure']

# Apparent temperature anomaly
df['apparent_temp_anomaly']   = df['apparent_temperature'] - df['air_temperature_2m']
df['apparent_air_temp_ratio'] = df['apparent_temperature'] / (df['air_temperature_2m'] + 1)

if 'lifted_index' in df.columns:
    df['lifted_index_negative'] = (-df['lifted_index']).clip(lower=0)

## Feature Engineering — Momentum & Lags

In [6]:
# ── Weather Momentum & Lag Features (NO target leakage) ──────────────────────
weather_cols = ['wind_speed_80m', 'solar_forecast', 'load_forecast',
                'wind_forecast', 'air_temperature_2m']

for col in weather_cols:
    grp = df.groupby('market_id')[col]
    df[f'{col}_diff_1h']  = grp.diff(1)
    df[f'{col}_diff_3h']  = grp.diff(3)
    df[f'{col}_diff_6h']  = grp.diff(6)
    df[f'{col}_diff_12h'] = grp.diff(12)
    df[f'{col}_rolling_mean_6h']  = grp.transform(lambda x: x.rolling(6,  min_periods=1).mean())
    df[f'{col}_rolling_std_6h']   = grp.transform(lambda x: x.rolling(6,  min_periods=1).std().fillna(0))
    df[f'{col}_rolling_mean_24h'] = grp.transform(lambda x: x.rolling(24, min_periods=1).mean())
    df[f'{col}_rolling_std_24h']  = grp.transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))
    df[f'{col}_rolling_min_24h']  = grp.transform(lambda x: x.rolling(24, min_periods=1).min().bfill())
    df[f'{col}_rolling_max_24h']  = grp.transform(lambda x: x.rolling(24, min_periods=1).max().bfill())
    df[f'{col}_range_24h']        = df[f'{col}_rolling_max_24h'] - df[f'{col}_rolling_min_24h']
    df[f'{col}_ewm_6h']   = grp.transform(lambda x: x.ewm(span=6,  adjust=False).mean())
    df[f'{col}_ewm_24h']  = grp.transform(lambda x: x.ewm(span=24, adjust=False).mean())
    df[f'{col}_zscore_24h'] = (df[col] - df[f'{col}_rolling_mean_24h']) / (df[f'{col}_rolling_std_24h'] + 0.001)

# Temperature anomaly vs recent history
df['temp_24h_mean']    = df.groupby('market_id')['air_temperature_2m'].transform(lambda x: x.rolling(24, min_periods=1).mean())
df['temp_72h_mean']    = df.groupby('market_id')['air_temperature_2m'].transform(lambda x: x.rolling(72, min_periods=1).mean())
df['temp_anomaly_24h'] = df['air_temperature_2m'] - df['temp_24h_mean']
df['temp_anomaly_72h'] = df['air_temperature_2m'] - df['temp_72h_mean']

# Wind direction
df['wind_dir_sin']    = np.sin(np.deg2rad(df['wind_direction_80m']))
df['wind_dir_cos']    = np.cos(np.deg2rad(df['wind_direction_80m']))
df['wind_dir_change'] = df.groupby('market_id')['wind_direction_80m'].diff(1).abs()

# Pressure & humidity interactions
df['pressure_temp_interaction'] = df['surface_pressure'] * df['air_temperature_2m']
df['humidity_temp_interaction'] = df['relative_humidity_2m'] * df['air_temperature_2m']
df['pressure_gradient']         = df.groupby('market_id')['surface_pressure'].diff(1)

# Cloud & precipitation
df['cloud_cover_total_sq'] = df['cloud_cover_total'] ** 2
df['cloud_cover_effect']   = df['cloud_cover_total'] * df['global_horizontal_irradiance']
df['precip_prob_sq']       = df['precipitation_probability'] ** 2
df['precip_effect']        = df['precipitation'] * df['precipitation_probability']

# Radiation efficiency
df['solar_efficiency']      = df['solar_forecast'] / (df['global_horizontal_irradiance'] + 1)
df['radiation_cloud_ratio'] = df['global_horizontal_irradiance'] / (df['cloud_cover_total'] + 1)

## Feature Engineering — Temporal Patterns

In [7]:
# ── Advanced Temporal Features ───────────────────────────────────────────────
df['hour_from_peak']   = abs(df['hour'] - 12)
df['is_peak_solar']    = ((df['hour'] >= 10) & (df['hour'] <= 16)).astype(int)
df['is_off_peak']      = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
df['is_business_hours']= ((df['hour'] >= 8) & (df['hour'] <= 18) & (df['day_of_week'] < 5)).astype(int)

df['is_monday']       = (df['day_of_week'] == 0).astype(int)
df['is_friday']       = (df['day_of_week'] == 4).astype(int)
df['is_weekend_start']= (df['day_of_week'] == 4).astype(int)
df['is_weekend_end']  = (df['day_of_week'] == 6).astype(int)

df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
df['is_spring'] = df['month'].isin([3, 4, 5]).astype(int)
df['is_autumn'] = df['month'].isin([9, 10, 11]).astype(int)

df['q1_temp_interaction'] = (df['quarter'] == 1) * df['air_temperature_2m']
df['q2_temp_interaction'] = (df['quarter'] == 2) * df['air_temperature_2m']
df['q3_temp_interaction'] = (df['quarter'] == 3) * df['air_temperature_2m']
df['q4_temp_interaction'] = (df['quarter'] == 4) * df['air_temperature_2m']

df['winter_load_factor'] = df['is_winter'] * df['load_forecast']
df['summer_load_factor'] = df['is_summer'] * df['load_forecast']
df['spring_load_factor'] = df['is_spring'] * df['load_forecast']
df['autumn_load_factor'] = df['is_autumn'] * df['load_forecast']

## Feature Engineering — Historical Target Encoding

In [8]:
# ── Historical Target Encoding (Validation-Safe) ─────────────────────────────
strict_train = df[(df['is_test'] == 0) & (df['delivery_start'] < VAL_PHYSICS_START)]

mean_mh  = strict_train.groupby(['market_id','hour'])['target'].mean().reset_index(name='target_histmean_mh')
mean_mdow= strict_train.groupby(['market_id','day_of_week'])['target'].mean().reset_index(name='target_histmean_mdow')
mean_mm  = strict_train.groupby(['market_id','month'])['target'].mean().reset_index(name='target_histmean_mm')
mean_m   = strict_train.groupby(['market_id'])['target'].mean().reset_index(name='target_histmean_m')
mean_h   = strict_train.groupby(['hour'])['target'].mean().reset_index(name='target_histmean_h')
mean_mhd = strict_train.groupby(['market_id','hour','day_of_week'])['target'].mean().reset_index(name='target_histmean_mhd')
mean_mq  = strict_train.groupby(['market_id','quarter'])['target'].mean().reset_index(name='target_histmean_mq')

df = df.merge(mean_mh,  on=['market_id','hour'],              how='left')
df = df.merge(mean_mdow,on=['market_id','day_of_week'],       how='left')
df = df.merge(mean_mm,  on=['market_id','month'],             how='left')
df = df.merge(mean_m,   on=['market_id'],                     how='left')
df = df.merge(mean_h,   on=['hour'],                          how='left')
df = df.merge(mean_mhd, on=['market_id','hour','day_of_week'],how='left')
df = df.merge(mean_mq,  on=['market_id','quarter'],           how='left')

global_mean = strict_train['target'].mean()
for c in [c for c in df.columns if c.startswith('target_histmean_')]:
    df[c] = df[c].fillna(global_mean)

df['histmean_mh_x_residual']      = df['target_histmean_mh'] * df['residual_demand']
df['histmean_mh_x_tightness']     = df['target_histmean_mh'] * df['tightness_ratio']
df['histmean_deviation_dow_vs_m'] = df['target_histmean_mdow'] - df['target_histmean_m']
df['histmean_deviation_mh_vs_h']  = df['target_histmean_mh']  - df['target_histmean_h']

print(f'✅ Safe historical encoding: {sum(c.startswith("target_histmean") for c in df.columns)} features')
print(f'   Global training mean: {global_mean:.4f}')

✅ Safe historical encoding: 7 features
   Global training mean: 37.3241


## Feature Engineering — Interactions & Final NaN Fill

In [9]:
# ── Interaction features ─────────────────────────────────────────────────────
df['temp_load_interaction']  = df['air_temperature_2m'] * df['load_forecast']
df['wind_load_interaction']  = df['wind_speed_80m']     * df['load_forecast']
df['solar_load_interaction'] = df['solar_forecast']     * df['load_forecast']
df['temp_wind_interaction']  = df['air_temperature_2m'] * df['wind_speed_80m']
df['temp_solar_interaction'] = df['air_temperature_2m'] * df['solar_forecast']
df['wind_solar_interaction'] = df['wind_speed_80m']     * df['solar_forecast']

df['temp_wind_load_interaction']  = df['air_temperature_2m'] * df['wind_speed_80m'] * df['load_forecast']
df['temp_solar_load_interaction'] = df['air_temperature_2m'] * df['solar_forecast'] * df['load_forecast']
df['wind_solar_load_interaction'] = df['wind_speed_80m']     * df['solar_forecast'] * df['load_forecast']

df['temp_volatility']  = df.groupby('market_id')['air_temperature_2m'].transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))
df['wind_volatility']  = df.groupby('market_id')['wind_speed_80m'].transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))
df['solar_volatility'] = df.groupby('market_id')['solar_forecast'].transform(lambda x: x.rolling(24, min_periods=1).std().fillna(0))

df['temp_rate_change']  = df.groupby('market_id')['air_temperature_2m'].diff(1) / (df.groupby('market_id')['air_temperature_2m'].shift(1).abs() + 0.01)
df['wind_rate_change']  = df.groupby('market_id')['wind_speed_80m'].diff(1)     / (df.groupby('market_id')['wind_speed_80m'].shift(1).abs() + 0.01)
df['solar_rate_change'] = df.groupby('market_id')['solar_forecast'].diff(1)     / (df.groupby('market_id')['solar_forecast'].shift(1).abs() + 0.01)

for col in ['wind_speed_80m', 'solar_forecast', 'load_forecast']:
    ts_mean = df.groupby('delivery_start')[col].transform('mean')
    ts_std  = df.groupby('delivery_start')[col].transform('std') + 0.001
    df[f'{col}_market_diff']   = df[col] - ts_mean
    df[f'{col}_market_zscore'] = (df[col] - ts_mean) / ts_std

for col in ['wind_speed_80m', 'solar_forecast', 'load_forecast']:
    df[f'{col}_skew_24h'] = df.groupby('market_id')[col].transform(
        lambda x: x.rolling(24, min_periods=12).skew().fillna(0))

# Summer heatwave & grid stress
df['heatwave_stress'] = np.where(
    (df['air_temperature_2m'] > 25) & (df['wind_speed_80m'] < 5),
    (df['air_temperature_2m'] - 25) ** 2, 0)
df['renewable_drought'] = ((df['solar_forecast'] < 10) & (df['wind_forecast'] < 10)).astype(int)
df['cooling_degree_load'] = df['load_forecast'] * np.maximum(0, df['air_temperature_2m'] - 22)

# Winter physics & grid stress
df['heating_degree_load'] = df['load_forecast'] * np.maximum(0, 18 - df['air_temperature_2m'])
df['wind_cutout_risk']    = (df['wind_speed_80m'] > 22).astype(int)
df['wind_cutout_penalty'] = df['wind_cutout_risk'] * df['residual_demand']

df['dark_cold_stress'] = np.where(
    (df['air_temperature_2m'] < 5) & (df['solar_forecast'] < 10) & (df['wind_speed_80m'] < 5),
    (5 - df['air_temperature_2m']) * df['load_forecast'], 0)

df['wind_chill_proxy'] = np.where(
    df['air_temperature_2m'] < 10,
    df['air_temperature_2m'] - (df['wind_speed_80m'] * 0.5),
    df['air_temperature_2m'])

# Final NaN fill
exclude_from_fill = {'target', 'delivery_start', 'delivery_end', 'market', 'id'}
for col in df.columns:
    if col in exclude_from_fill: continue
    if df[col].dtype in ['float64', 'float32', 'int64', 'int32', 'int8']:
        nan_count = df[col].isna().sum()
        if nan_count > 0:
            df[col] = df[col].fillna(df[col].median())

print(f'✅ Feature engineering complete: {len(df.columns)} total columns')
print(f'   Training rows: {(df["is_test"]==0).sum()}, Test rows: {(df["is_test"]==1).sum()}')

✅ Feature engineering complete: 240 total columns
   Training rows: 132608, Test rows: 13098


## MOD 1: Feature Pruning (Useless + Correlated > 0.98)

In [10]:
# ── MOD 1: Feature Pruning ───────────────────────────────────────────────────
print('--- PRUNING USELESS FEATURES ---')
useless_features = [
    'precip_prob_sq', 'precipitation', 'load_forecast_market_zscore',
    'load_forecast_market_diff', 'extreme_temp', 'extreme_precip',
    'is_off_peak', 'heatwave_stress', 'dark_cold_stress', 'precip_effect',
    'quarter', 'solar_forecast_rolling_min_24h', 'renewable_drought',
    'precipitation_amount', 'precipitation_probability', 'is_summer',
    'is_autumn', 'is_monday', 'hour_from_peak', 'is_peak_solar'
]
actual_drops = [c for c in useless_features if c in df.columns]
df.drop(columns=actual_drops, inplace=True)
print(f'  Dropped {len(actual_drops)} useless features')

print('\n--- PRUNING CORRELATED FEATURES ---')
numeric_cols = df.select_dtypes(include=[np.number]).columns
if 'target' in numeric_cols: numeric_cols = numeric_cols.drop('target')
corr_matrix = df[numeric_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.98)]
df.drop(columns=to_drop_corr, inplace=True)
print(f'  Dropped {len(to_drop_corr)} correlated features')
print(f'  Remaining columns: {len(df.columns)}')

--- PRUNING USELESS FEATURES ---
  Dropped 20 useless features

--- PRUNING CORRELATED FEATURES ---
  Dropped 33 correlated features
  Remaining columns: 187


## MOD 2+3: Unified Train/Val Split + Categorical Casting + Winter Weights

In [11]:
# ── MOD 2+3: Unified Split + Categorical Columns + Winter Weights ────────────
observed_df = df[df['is_test'] == 0].copy()
test_df     = df[df['is_test'] == 1].copy()

mask_val_physics = (observed_df['delivery_start'] >= VAL_PHYSICS_START) & (observed_df['delivery_start'] < VAL_PHYSICS_END)
mask_val_recency = (observed_df['delivery_start'] >= VAL_RECENCY_START)
mask_train       = ~(mask_val_physics | mask_val_recency)

train_df       = observed_df[mask_train]
val_physics_df = observed_df[mask_val_physics]

drop_cols = {'id', 'target', 'market', 'delivery_start', 'delivery_end', 'is_test'}
feat_cols = sorted([c for c in df.columns if c not in drop_cols])

# Declare categoricals explicitly (only those remaining after pruning)
cat_cols = [c for c in ['market_id', 'hour', 'day_of_week', 'month'] if c in feat_cols]
print(f'  Categorical columns: {cat_cols}')

X_train       = train_df[feat_cols].copy()
y_train_real  = train_df['target'].values
y_train       = np.arcsinh(train_df['target'].values)

X_val_physics      = val_physics_df[feat_cols].copy()
y_val_physics_real = val_physics_df['target'].values
y_val_physics      = np.arcsinh(val_physics_df['target'].values)

X_all  = observed_df[feat_cols].copy()
y_all  = np.arcsinh(observed_df['target'].values)

X_test = test_df[feat_cols].copy()

# Cast categoricals to 'category' dtype
for ds in [X_train, X_val_physics, X_all, X_test]:
    for c in cat_cols:
        ds[c] = ds[c].astype('category')

# MOD 3: Winter weights — months 9-12 get 3x weight
train_month_vals = train_df['month'].values
train_weights = np.where(np.isin(train_month_vals, [9, 10, 11, 12]), 3.0, 1.0)

all_month_vals = observed_df['month'].values
all_weights = np.where(np.isin(all_month_vals, [9, 10, 11, 12]), 3.0, 1.0)

print(f'🌲 X_train:       {len(X_train):,}')
print(f'❄️ X_val_physics: {len(X_val_physics):,}')
print(f'📦 X_all:         {len(X_all):,}')
print(f'🔬 X_test:        {len(X_test):,}')
print(f'⚖️ Winter-weighted rows (train): {(train_weights==3.0).sum():,}')

  Categorical columns: ['market_id', 'hour', 'day_of_week', 'month']
🌲 X_train:       101,792
❄️ X_val_physics: 17,568
📦 X_all:         132,608
🔬 X_test:        13,098
⚖️ Winter-weighted rows (train): 15,769


## BLOCK A — LightGBM Optuna (Unified + Winter Weights)

In [12]:
# ── BLOCK A: LightGBM Optuna (Unified + Winter Weights) ──────────────────────
t0 = time.time()
print('='*60); print('BLOCK A: LightGBM'); print('='*60)

cat_idx_lgb = [feat_cols.index(c) for c in cat_cols]

def _lgb_obj(trial):
    p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
         'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
         'num_leaves':    trial.suggest_int('nl', 31, 512),
         'alpha':         trial.suggest_float('alpha', 1.0, 3.0)}
    ds_t = lgb.Dataset(X_train, y_train, categorical_feature=cat_idx_lgb,
                        weight=train_weights, free_raw_data=False)
    ds_v = lgb.Dataset(X_val_physics, y_val_physics, reference=ds_t, free_raw_data=False)
    m = lgb.train(p, ds_t, 2000, valid_sets=[ds_v],
                  callbacks=[lgb.early_stopping(50, verbose=False)])
    trial.set_user_attr('bi', m.best_iteration)
    preds = np.sinh(np.clip(m.predict(X_val_physics), -20, 20))
    return root_mean_squared_error(y_val_physics_real, preds)

s_lgb = optuna.create_study(direction='minimize')
s_lgb.optimize(_lgb_obj, n_trials=N_TRIALS)
print(f'  Best LGB RMSE={s_lgb.best_value:.4f}')

bi_lgb = s_lgb.best_trial.user_attrs['bi']
_p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
      'learning_rate': s_lgb.best_params['lr'],
      'num_leaves':    s_lgb.best_params['nl'],
      'alpha':         s_lgb.best_params['alpha']}
val_lgb = lgb.train(_p,
    lgb.Dataset(X_train, y_train, categorical_feature=cat_idx_lgb, weight=train_weights),
    bi_lgb)
val_lgb.save_model('../models/v14/lgb_val.txt')
json.dump({'params': s_lgb.best_params, 'bi': bi_lgb},
          open('../models/v14/lgb_params.json', 'w'))
print(f'  LGB done in {time.time()-t0:.0f}s')

BLOCK A: LightGBM
  Best LGB RMSE=47.4800
  LGB done in 1272s


## BLOCK B — XGBoost Optuna (Unified + Winter Weights + Safe Huber)

In [13]:
# ── BLOCK B: XGBoost Optuna (Unified + Winter Weights + Safe Huber) ──────────
t0 = time.time()
print('='*60); print('BLOCK B: XGBoost'); print('='*60)

# XGBoost needs numeric categories
def _to_xgb(X):
    Xc = X.copy()
    for c in cat_cols:
        Xc[c] = Xc[c].cat.codes
    return Xc

dt_xgb = xgb.DMatrix(_to_xgb(X_train), label=y_train, weight=train_weights)
dv_xgb = xgb.DMatrix(_to_xgb(X_val_physics), label=y_val_physics)

def _xgb_obj(trial):
    p = {
        'tree_method': 'hist',
        'objective':   'reg:pseudohubererror',
        'eval_metric': 'rmse',
        'huber_slope': trial.suggest_float('alpha', 1.0, 3.0),
        'seed':        SEED,
        'n_jobs':      -1,
        'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
        'max_depth':     trial.suggest_int('md', 5, 12)
    }
    m = xgb.train(p, dt_xgb, 2000, evals=[(dv_xgb,'v')],
                  early_stopping_rounds=50, verbose_eval=False)
    trial.set_user_attr('bi', m.best_iteration)
    preds = np.sinh(np.clip(m.predict(dv_xgb), -20, 20))
    return root_mean_squared_error(y_val_physics_real, preds)

s_xgb = optuna.create_study(direction='minimize')
s_xgb.optimize(_xgb_obj, n_trials=N_TRIALS)
print(f'  Best XGB RMSE={s_xgb.best_value:.4f}')

bi_xgb = s_xgb.best_trial.user_attrs['bi']
_p = {'tree_method':'hist','objective':'reg:pseudohubererror',
      'eval_metric':'rmse','seed':SEED,'n_jobs':-1,
      'learning_rate': s_xgb.best_params['lr'],
      'max_depth':     s_xgb.best_params['md'],
      'huber_slope':   s_xgb.best_params['alpha']}
val_xgb = xgb.train(_p, dt_xgb, bi_xgb, evals=[(dv_xgb,'v')], verbose_eval=False)
val_xgb.save_model('../models/v14/xgb_val.json')
json.dump({'params': s_xgb.best_params, 'bi': bi_xgb},
          open('../models/v14/xgb_params.json', 'w'))
print(f'  XGB done in {time.time()-t0:.0f}s')

BLOCK B: XGBoost
  Best XGB RMSE=46.6466
  XGB done in 3666s


## BLOCK C — CatBoost Optuna (Unified + Winter Weights)

In [14]:
# ── BLOCK C: CatBoost Optuna (Unified + Winter Weights) ──────────────────────
t0 = time.time()
print('='*60); print('BLOCK C: CatBoost'); print('='*60)

cat_feat_idx = [list(X_train.columns).index(c) for c in cat_cols]

def _cat_obj(trial):
    p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','eval_metric':'RMSE',
         'random_seed':SEED,'verbose':False,'iterations':1000,'early_stopping_rounds':50,
         'learning_rate': trial.suggest_float('lr', 1e-3, 0.1, log=True),
         'depth':         trial.suggest_int('depth', 4, 10)}
    m = CatBoostRegressor(**p)
    m.fit(Pool(X_train, y_train, cat_features=cat_feat_idx, weight=train_weights),
          eval_set=Pool(X_val_physics, y_val_physics, cat_features=cat_feat_idx),
          use_best_model=True)
    trial.set_user_attr('bi', m.get_best_iteration())
    preds = np.sinh(np.clip(m.predict(X_val_physics), -20, 20))
    return root_mean_squared_error(y_val_physics_real, preds)

s_cat = optuna.create_study(direction='minimize')
s_cat.optimize(_cat_obj, n_trials=15)
print(f'  Best CAT RMSE={s_cat.best_value:.4f}')

bi_cat = s_cat.best_trial.user_attrs['bi']
_p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','eval_metric':'RMSE',
      'random_seed':SEED,'verbose':False,'iterations':bi_cat,
      'learning_rate': s_cat.best_params['lr'],
      'depth':         s_cat.best_params['depth']}
val_cat = CatBoostRegressor(**_p)
val_cat.fit(Pool(X_train, y_train, cat_features=cat_feat_idx, weight=train_weights),
            eval_set=Pool(X_val_physics, y_val_physics, cat_features=cat_feat_idx),
            use_best_model=True)
val_cat.save_model('../models/v14/cat_val.cbm')
json.dump({'params': s_cat.best_params, 'bi': bi_cat},
          open('../models/v14/cat_params.json', 'w'))
print(f'  CAT done in {time.time()-t0:.0f}s')

BLOCK C: CatBoost
  Best CAT RMSE=49.2251
  CAT done in 2228s


## BLOCK D — SciPy Global Ensemble Weight Optimization

In [15]:
# ── BLOCK D: SciPy Global Ensemble Weight Optimization ───────────────────────
print('='*60); print('BLOCK D: SCIPY WEIGHT OPTIMIZATION'); print('='*60)

# Predict on full validation set using unified val models
# All predictions wrapped in np.sinh(np.clip(..., -20, 20))
vp_lgb = np.sinh(np.clip(val_lgb.predict(X_val_physics), -20, 20))
vp_xgb = np.sinh(np.clip(val_xgb.predict(xgb.DMatrix(_to_xgb(X_val_physics))), -20, 20))
vp_cat = np.sinh(np.clip(val_cat.predict(X_val_physics), -20, 20))

def _blend_rmse(w, p1, p2, p3, y_real):
    return root_mean_squared_error(y_real, w[0]*p1 + w[1]*p2 + w[2]*p3)

cons = {'type':'eq','fun': lambda w: w.sum()-1.0}
bnd  = [(0, 1)] * 3
w0   = [1/3, 1/3, 1/3]

res = sp_minimize(_blend_rmse, w0,
                  args=(vp_lgb, vp_xgb, vp_cat, y_val_physics_real),
                  method='SLSQP', bounds=bnd, constraints=cons)

best_w = res.x
print(f'  Global weights [LGB, XGB, CAT]: {np.round(best_w, 4)}')
print(f'  Val Physics RMSE: {res.fun:.4f}')
json.dump({'weights': best_w.tolist()},
          open('../models/v14/ensemble_weights.json', 'w'))

BLOCK D: SCIPY WEIGHT OPTIMIZATION
  Global weights [LGB, XGB, CAT]: [0.129 0.871 0.   ]
  Val Physics RMSE: 46.6241


## BLOCK E — Full Retraining on All Observed Data

In [16]:
# ── BLOCK E: Full Retraining on All Observed Data (Unified + Winter Weights) ──
print('='*60); print('BLOCK E: FULL RETRAINING'); print('='*60)

cat_idx_lgb_all = [list(X_all.columns).index(c) for c in cat_cols]

# ── LGB final ──
_p = {'objective':'huber','metric':'rmse','verbosity':-1,'seed':SEED,'n_jobs':-1,
      'learning_rate': s_lgb.best_params['lr'],
      'num_leaves':    s_lgb.best_params['nl'],
      'alpha':         s_lgb.best_params['alpha']}
lgb_final = lgb.train(_p,
    lgb.Dataset(X_all, y_all, categorical_feature=cat_idx_lgb_all, weight=all_weights),
    num_boost_round=int(bi_lgb * 1.1))
lgb_final.save_model('../models/v14/lgb_final.txt')

# ── XGB final ──
da_xgb = xgb.DMatrix(_to_xgb(X_all), label=y_all, weight=all_weights)
_p = {'tree_method':'hist','objective':'reg:pseudohubererror',
      'eval_metric':'rmse','seed':SEED,'n_jobs':-1,
      'learning_rate': s_xgb.best_params['lr'],
      'max_depth':     s_xgb.best_params['md'],
      'huber_slope':   s_xgb.best_params['alpha']}
xgb_final = xgb.train(_p, da_xgb, num_boost_round=int(bi_xgb * 1.1))
xgb_final.save_model('../models/v14/xgb_final.json')

# ── CAT final ──
_p = {'loss_function':'Huber:delta=1.5','task_type':'CPU','random_seed':SEED,'verbose':False,
      'learning_rate': s_cat.best_params['lr'],
      'depth':         s_cat.best_params['depth'],
      'iterations':    int(bi_cat * 1.1)}
cat_final = CatBoostRegressor(**_p)
cat_final.fit(Pool(X_all, y_all, cat_features=cat_feat_idx, weight=all_weights))
cat_final.save_model('../models/v14/cat_final.cbm')

print('  All 3 unified final models saved.')

BLOCK E: FULL RETRAINING
  All 3 unified final models saved.


## BLOCK F — Ensemble Inference & Submission

In [17]:
# ── BLOCK F: Ensemble Inference & Submission ──────────────────────────────────
print('='*60); print('BLOCK F: ENSEMBLE INFERENCE'); print('='*60)

# LGB predictions on test
tst_lgb = np.sinh(np.clip(lgb_final.predict(X_test), -20, 20))

# XGB predictions on test
tst_xgb = np.sinh(np.clip(xgb_final.predict(xgb.DMatrix(_to_xgb(X_test))), -20, 20))

# CAT predictions on test
tst_cat = np.sinh(np.clip(cat_final.predict(X_test), -20, 20))

# Combine using global SciPy-optimal weights
final_preds = best_w[0]*tst_lgb + best_w[1]*tst_xgb + best_w[2]*tst_cat
# Safety re-clip on combined predictions
final_preds = np.sinh(np.clip(np.arcsinh(final_preds), -20, 20))

# Build submission
pred_df    = pd.DataFrame({'id': test_df['id'].values, 'target': final_preds})
submission = sample_sub[['id']].merge(pred_df, on='id', how='left')

assert len(submission) == len(sample_sub)
assert submission['target'].isna().sum() == 0
assert (submission['id'] == sample_sub['id']).all()

path = '../submissions/submission_v14_ultimate_unified_ensemble.csv'
submission.to_csv(path, index=False)

print('✅ Saved:', path)
print('   Mean:  ', round(submission['target'].mean(), 4))
print('   Std:   ', round(submission['target'].std(),  4))
print('   Min:   ', round(submission['target'].min(),  4))
print('   Max:   ', round(submission['target'].max(),  4))
print('   Median:', round(submission['target'].median(), 4))

BLOCK F: ENSEMBLE INFERENCE
✅ Saved: ../submissions/submission_v14_ultimate_unified_ensemble.csv
   Mean:   29.9968
   Std:    25.5895
   Min:    -22.6493
   Max:    259.4966
   Median: 26.912
